## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"

## Этап 3. Оффлайн-рекомендации

### Постановка задачи

Подготовьте данные для обучения:
- Загрузите ранее сохранённые данные.
- Разбейте данные на тренировочную и тестовую выборки: в тренировочную отнесите все данные до __16 декабря 2022 г. (не включительно)__.

Рассчитайте рекомендации нескольких типов, сохраните каждый тип в своём файле:
- топ популярных — в `top_popular.parquet`,
- персональные (при помощи ALS) — в `personal_als.parquet`,
- похожие треки или i2i (при помощи ALS) — в `similar.parquet`.

Постройте ранжирующую модель, которая использует минимум три признака. Отранжируйте рекомендации, а итоговые сохраните в `recommendations.parquet`.

Проверьте оценку качества трёх типов рекомендаций: 
1. топ популярных,
2. персональных, полученных при помощи ALS,
3. итоговых (по метрикам: recall, precision, coverage, novelty).

Приведите значения метрик в README.

### Инициализация

In [1]:
import os
import numpy as np
import pandas as pd
import joblib
import ipywidgets

from sklearn.metrics import accuracy_score
import scipy
import sklearn.preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from catboost import CatBoostClassifier, Pool

In [2]:
import sys

# Проверяем, в каком окружении работаем
print(sys.executable)

/home/mle-user/.venv/bin/python


In [2]:
# Отключаем предупреждения
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning, FutureWarning))

### Загрузка и разбиение данных

In [3]:
# Загружаем основные данные
items = pd.read_parquet("items.parquet")
events = pd.read_parquet("events.parquet")

Разбиваем данные на тренировочную и тестовую выборки (в последнюю относим данные до 16 декабря 2022 г. не включительно).

In [4]:
# Зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2022-12-16")

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

### Рекомендации на основе самых популярных треков

Загружаем ранее сохраненные топ-треки и оцениваем качество рекомендаций на их основе для "холодных" пользователей.

In [6]:
# Загружаем топ-100 самых прослушиваемых треков
top_k_pop_items = pd.read_parquet("top_popular.parquet")
top_k_pop_items.head()

,item_id,plays,users,name,genres,artists,albums
0,53404,15704,15704,Smells Like Teen Spirit,"[alternative, allrock, rock]",[Nirvana],[Smells Like Teen Spirit / In Bloom / On A Pla...
1,178529,14088,14088,Numb,"[numetal, metal]",[Linkin Park],"[Meteora, 00s Rock Anthems]"
2,37384,12798,12798,Zombie,"[allrock, rock]",[The Cranberries],"[90s Alternative, MNM Sing Your Song: Back To ..."
3,6705392,12735,12735,Seven Nation Army,[alternative],[The White Stripes],"[Pay Close Attention : XL Recordings, Radio 1 ..."
4,33311009,11972,11972,Believer,"[allrock, rock]",[Imagine Dragons],"[Horoscope Tunes: Cancer, Workout Music Hits 2..."


In [7]:
# Идентификаторы уникальных пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()

In [12]:
# Идентификаторы "холодных" пользователей
cold_users = users_test[~users_test.isin(users_train)]
print(f"Кол-во холодных пользователей: {len(cold_users)}")

Series([], Name: user_id, dtype: int32)

Удаление большого кол-ва записей из истории взаимодействий привело к тому, что холодных пользователей не оказалось.

In [9]:
# Формируем бинарные колонки target и prediction

top_k_pop_items['prediction'] = 1

cold_users_events = \
    events_test[events_test["user_id"].isin(cold_users)] \
        .merge(top_k_pop_items[['item_id', 'prediction']], on="item_id", how="left")

cold_users_events['prediction'].fillna(0, inplace=True)

cold_users_events['target'] = 1

In [10]:
# Точность рекомендаций
accuracy = accuracy_score(cold_users_events['prediction'], cold_users_events['target'])
print(accuracy)

nan


/home/mle-user/.venv/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/mle-user/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [13]:
# Считаем покрытие "холодных" пользователей рекомендациями.
cold_users_hit_ratio = cold_users_events.groupby("user_id").agg(hits=("prediction", 'sum'))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: nan
Среднее покрытие пользователей: nan


### Персональные рекомендации на основе коллаборативного подхода и ALS

[Техническая документация на библиотеку implicit](https://benfred.github.io/implicit/api/models/cpu/als.html)


Перекодируем идентификаторы `user_id` и `item_id`.

In [5]:
# Перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# Перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train = events_train.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left')
events_test = events_test.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left')

Создаем sparse-матрицу в формате [CSR](https://docs.scipy.org/doc/scipy/reference/sparse.html)

In [15]:
# Формируем колонку target (1 - трек прослушан, 0 - нет)
events_train['target'] = 1

# Создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["target"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

Имея подготовленную матрицу взаимодействий, создаем ALS-модель. Для примера возьмём количество латентных факторов для матриц $P и Q$ равным 50. 

In [16]:
# Код для создания и тренировки модели
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/.venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

In [18]:
# Сохраняем als-модель в файл
os.makedirs('models/', exist_ok=True)
with open('models/als_model.pkl', 'wb') as fd:
    joblib.dump(als_model, fd)

Чтобы получить рекомендации для отдельного пользователя с помощью модели ALS, используем функцию ниже.

In [19]:
def get_recommendations_als(user_item_matrix, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = als_model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

Посмотрим историю и рекомендации для случайного пользователя.

In [22]:
# Выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]
print(f"user_id: {user_id}")

print("Последние события:")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
)
user_history_to_print = user_history[["item_id", "name", "genres", "albums", "artists"]].tail(10)
display(user_history_to_print)

print("Рекомендации:")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=10)
user_recommendations = user_recommendations.merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
user_recommendations_to_print = \
    user_recommendations[["item_id", "name", "genres", "albums", "artists", "score"]].head(10)
display(user_recommendations_to_print)

user_id: 1160915
Последние события:


,item_id,name,genres,albums,artists
976,69978738,Бро не женись пока нет,"[rusrap, rap]","[Бро не женись пока нет, Дорога в Сити. Deluxe...",[Нурминский]
977,70103692,Дотянись,"[rusrap, rap]",[Дотянись],[Tengri]
978,70435565,Every Time We Touch,"[allrock, rock]","[Covers, Vol. 3, Every Time We Touch, Cole Rol...","[Lauren Babic, Cole Rolland]"
979,70488657,New Divide,"[allrock, rock]","[Young's Old Covers (2019-2021), New Divide]",[Jonathan Young]
980,70764600,Имя твоё,"[metal, classicmetal]",[Имя твоё],"[Юрий Суздаль, Колизей]"
981,70940455,Нет времени для тебя,"[None, metal]",[Нет времени для тебя],[Трибунал]
982,71018795,Дом-на-крови,"[numetal, metal]",[Начало нового круга],[LOUNA]
983,71018799,Молчание ягнят,"[numetal, metal]",[Начало нового круга],[LOUNA]
984,71238337,Пацанам,"[rusrap, rap]",[Пацанам],[Литвиненко]
985,71503290,Большой дядя,"[rusrap, rap]","[Дорога в Сити. Deluxe Version, Дорога в Сити]",[Нурминский]


Рекомендации:


,item_id,name,genres,albums,artists,score
0,418017,Hero,[alternative],"[Awake, WOW Hits 2011, Vital Signs]",[Skillet],0.649034
1,135147,Comatose,[alternative],"[Vital Signs, Comatose]",[Skillet],0.647248
2,27032843,Shut Your Mouth,"[industrial, metal]","[Nothing Remains The Same, Best Death Metal Ba...",[Pain],0.641140
3,328683,Bring Me To Life,"[alternative, allrock, rock]","[Mental Health 2020, Monster Millennium Hits, ...",[Evanescence],0.635593
4,178529,Numb,"[numetal, metal]","[Meteora, 00s Rock Anthems]",[Linkin Park],0.628167
5,29456722,They Don't Care About Us,"[numetal, metal]","[Love, Lies & Therapy]",[Saliva],0.621304
6,21709897,Still Waiting,[alternative],[All the Good Shit: 14 Solid Gold Hits 2000-2008],[Sum 41],0.619633
7,178495,In the End,"[numetal, metal]","[Antyradio: Najlepszy Rock Na Swiecie Vol. 4, ...",[Linkin Park],0.606558
8,14701552,How You Remind Me,"[allrock, rock]","[Playlist: Alt Rock, Rock Love, I Love Power B...",[Nickelback],0.603753
9,630670,"You're Gonna Go Far, Kid",[punk],"[You're Gonna Go Far, Kid, Sleep Music Rock Ed...",[The Offspring],0.602788


Получим рекомендации для всех имеющихся пользователей.

In [23]:
# Получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# Получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, 
    N=100)

Код возвращает рекомендации как список списков, что не очень удобно. Преобразуем его в более удобный табличный формат.

In [24]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [25]:
# Смотрим, что получилось
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.head()

In [34]:
# Сохраним полученные рекомендации в файл
als_recommendations.to_parquet("personal_als.parquet")

,user_id,item_id,score,target,prediction,played,rank
1964300,801589,51241318,3.114186,0.0,1,True,1
754900,307933,70408028,2.804013,0.0,1,True,1
2592900,1055078,18860,2.770963,0.0,1,True,1
730700,298960,9060176,2.690973,0.0,1,True,1
774600,316011,28420997,2.663807,0.0,1,True,1


Оценим точность рекомендаций на тестовой выборке с порогом вероятности 0.5.

In [26]:
# Формируем бинарные колонки target и prediction, для последней берем порог вероятности 0.5

events_test['target'] = 1

als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "target"]], on=["user_id", "item_id"], how="left")
)

als_recommendations['target'] = als_recommendations['target'].fillna(0)

als_recommendations['prediction'] = (als_recommendations['score'] >= 0.5).astype('int')

In [27]:
# Точность рекомендаций
accuracy = accuracy_score(als_recommendations['prediction'], als_recommendations['target'])
print(accuracy)

0.4865243305755716


Посчитаем несколько специфичных метрик. 

Найдем покрытие по объектам, используя все имеющиеся рекомендации.

In [28]:
cov_items = als_recommendations['item_id'].nunique() / len(items) 
print(cov_items)

0.010876911487221414


Посчитаем среднее Novelty@5, для этого:
- разметим каждую рекомендацию в als_recommendations булевым признаком played, используя events_train
(False — пользователь не прослушал трек, True — прослушал),
- посчитаем Novelty@5 для каждого пользователя,
- найдем среднее значение полученных Novelty@5.

In [29]:
# разметим каждую рекомендацию признаком played
events_train["played"] = True
als_recommendations = als_recommendations \
    .merge(events_train[['played', "user_id", "item_id"]], on=["user_id", "item_id"], how="left")
als_recommendations["played"] = als_recommendations["played"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by='score', ascending=False) 
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1 - als_recommendations.query("rank <= 5").groupby("user_id")["played"].mean())

# посчитаем средний novelty
novelty_5_mean = novelty_5.mean()
print(novelty_5_mean)

0.21277466417578508


Посчитаем recall и precision для ALS-рекомендаций. Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
- gt (ground truth): объект есть в тестовой выборке;
- pr (predicted): объект есть в рекомендациях.

Теперь разметим признаки бинарной классификации:
- TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
- FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
- FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

Создаем функцию для подсчета метрик TP, FP и FN у пользователей, которые есть одновременно и в events_train,
и в events_test.

In [28]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

Считаем TP, FP и FN для каждой пары user_id, item_id

In [31]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(events_train,
                                                                        events_test, 
                                                                        als_recommendations,
                                                                        top_k=5)

Common users: 24210


Считаем precision и recall

In [30]:
def compute_cls_metrics(events_recs_for_binary_metrics):
    
    groupper = events_recs_for_binary_metrics.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum()) 
    recall = recall.fillna(0).mean()

    return precision, recall

In [33]:
precision, recall = compute_cls_metrics(events_recs_for_binary_metrics)
print(round(precision, 3), round(recall, 3))

0.0 0.0


### Рекомендации на основе контентного подхода и ALS

Построим контентные рекомендации на основе принадлежности треков к различным жанрам.

Сначала загрузим список жанров по всем трекам, который мы создали на предыдущем этапе

In [35]:
genres = pd.read_parquet('genres.parquet')
genres.head()

,name,items_count,score
genre_id,,,
0,pop,166284,0.103221
1,folk,37547,0.023307
2,allrock,118506,0.073563
3,hardrock,10916,0.006776
4,rock,55174,0.034249


Функция ниже строит матрицу вида «трек-жанр»

In [36]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genres"] is None: # После предобработки NA быть не может, оставили для общего случая
            continue
        
        item_genres = list(v["genres"])
        
        for genre_name in item_genres:
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(1)
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), \
                                         shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанрам была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

Построим матрицу с весами по жанрам для каждого трека

In [37]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Будем искать рекомендации на основе косинусной близости. Функция ниже находит рекомендации для одного пользователя

In [39]:
def get_recommendations_by_genres(user_id, items, events, all_items_genres_csr, n=5):

    """ Возвращает n рекомендаций для user_id на основе предпочитаемых им жанров"""
    
    user_events = events.query("user_id == @user_id")
    user_items = items[items["item_id"].isin(user_events["item_id"])]

    user_items_genres_csr = all_items_genres_csr[user_items['item_id_enc']]
    
    # вычислим среднюю склонность пользователя к жанрам 
    user_genres_scores = np.asarray(user_items_genres_csr.mean(axis=0))

    # вычисляем сходство между треками пользователя и всеми имеющимися трекам
    similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

    # преобразуем в одномерный массив
    similarity_scores = similarity_scores.flatten()

    # получаем индексы top-n (по убыванию значений), т.е. encoded индексы самых похожих треков
    top_n_indices = np.argsort(-similarity_scores)[:n] 

    selected_items = items[items["item_id_enc"].isin(top_n_indices)]
    
    return selected_items['item_id'].to_list(), similarity_scores[top_n_indices].tolist()

Посмотрим рекомендации для произвольного пользователя

In [42]:
# Выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]
print(f"user_id: {user_id}")

print("Последние события:")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
)
user_history_to_print = user_history[["item_id", "name", "genres", "albums", "artists"]].tail(10)
display(user_history_to_print)

print("Рекомендации:")
selected_items, similarity_scores = \
    get_recommendations_by_genres(user_id, items, events_train, all_items_genres_csr, n=5)
selected_items = items[items["item_id"].isin(selected_items)]
selected_items['scores'] = similarity_scores

with pd.option_context("max_colwidth", 100):
   display(selected_items[["item_id", "name", "genres", "albums", "artists", 'scores']])

user_id: 672388
Последние события:


,item_id,name,genres,albums,artists
1868,99231063,Stand Up,"[numetal, metal]","[Ego Trip, Stand Up]",[Papa Roach]
1869,99233920,Close Your Eyes,"[electronics, dnb]","[Close Your Eyes, Cognition]","[Wilkinson, iiola]"
1870,100139128,Lost In The Grandeur,"[numetal, metal]","[Lost In The Grandeur, Requiem]",[Korn]
1871,100196196,Fiasco,[dance],[Technoblues Therapy (Part I)],[Intelligency]
1872,100196198,Instinct,[dance],[Technoblues Therapy (Part I)],[Intelligency]
1873,100199968,Let The Dark Do The Rest,"[numetal, metal]",[Requiem],[Korn]
1874,100199969,Disconnect,"[numetal, metal]",[Requiem],[Korn]
1875,100199971,Penance To Sorrow,"[numetal, metal]",[Requiem],[Korn]
1876,100199973,Worst Is On Its Way,"[numetal, metal]",[Requiem],[Korn]
1877,100846242,Fade Away,"[electronics, dnb]",[Cognition],"[Jem Cooke, Wilkinson, Pola & Bryson]"


Рекомендации:


,item_id,name,genres,albums,artists,scores
443651,31109251,Always,"[dance, electronics, pop]","[Jessica Jay Collection, JESSICA JAY, The Best of Jessica Jay]",[Jessica Jay],0.787798
443654,31109254,I'm Living,"[dance, electronics, pop]","[Jessica Jay Collection, JESSICA JAY, The Best of Jessica Jay]",[Jessica Jay],0.787798
443655,31109255,One More Time,"[dance, electronics, pop]","[Jessica Jay Collection, JESSICA JAY, The Best of Jessica Jay]",[Jessica Jay],0.787798
443656,31109256,The Room at the Top of the Stears,"[dance, electronics, pop]","[Jessica Jay Collection, JESSICA JAY, The Best of Jessica Jay]",[Jessica Jay],0.787798
443657,31109257,Broken Hearted Woman,"[dance, electronics, pop]","[Jessica Jay Collection, JESSICA JAY, The Best of Jessica Jay]",[Jessica Jay],0.787798


Запускаем поиск для пользователей из events_train. Для экономии ресурсов возьмем только 10000 первых user_id.

In [49]:
# Для примера взял только первые 1000 юзеров

users_ids = events_train['user_id'].unique()[:1000] 

n = 5
items_ids = []
scores = []
for i, user_id in tqdm(enumerate(users_ids), total=1000):
    user_items_ids, user_scores = \
        get_recommendations_by_genres(user_id, items, events_train, all_items_genres_csr, n)
    items_ids.append(user_items_ids)
    scores.append(user_scores)

    if i >= 1000:
        break

content_recommendations = pd.DataFrame({
    "user_id": users_ids,
    "item_id": items_ids, 
    "score": scores})

content_recommendations = content_recommendations.explode(["item_id", "score"], ignore_index=True)

100%|██████████| 1000/1000 [06:56<00:00,  2.40it/s]


In [50]:
# Смотрим результат
content_recommendations

,user_id,item_id,score
0,16,4879861,0.921282
1,16,32717047,0.921282
2,16,61326989,0.921282
3,16,61327379,0.921282
4,16,61331162,0.921282
...,...,...,...
4995,40745,16273834,0.841309
4996,40745,40694662,0.841309
4997,40745,45465132,0.841309
4998,40745,45495045,0.841309


In [51]:
# Сохраним полученные рекомендации в файл
content_recommendations.to_parquet("content_recommendations.parquet")

### Рекомендации на основе похожих треков (i2i) и ALS

Получим рекомендации при помощи метода [similar_items](https://benfred.github.io/implicit/api/models/cpu/als.html). Для экономии ресурсов возьмем не все объекты из `events_train`, а только первые 10 тыс.

In [ ]:
# Загружаем ALS-модель
with open('models/als_model.pkl', 'rb') as fd:
    als_model = joblib.load(fd)

In [57]:
# Получим энкодированные идентификаторы первых 10000 объектов в events_train
train_item_ids_enc = events_train['item_id_enc'].unique()[:10000]

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()}) 
similar_items = similar_items.explode(['sim_item_id_enc', 'score'], ignore_index=True) 

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype('int') 
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"]) 
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"]) 
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [59]:
# Смотрим результат
similar_items.head()

,score,item_id_1,item_id_2
1,0.924673,549,26771
2,0.923671,549,4198
3,0.921660,549,69658
4,0.921044,549,37402640
5,0.913545,549,1706463


In [60]:
# Сохраняем полученные рекомендации в файл
similar_items.to_parquet("similar.parquet")

С помощью функции ниже можно посмотреть рекомендации для одного трека

In [67]:
def print_sim_items(item_id, similar_items):
    item_columns_to_use = ["item_id", "name", "genres", "artists", "albums"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [68]:
# Задаем произвольный трек
item_id = similar_items['item_id_1'].sample().iat[0]

# Смотрим похожие треки
print_sim_items(item_id, similar_items)

,item_id,name,genres,artists,albums
297321,19197327,Hokage's Funeral (From 'Naruto'),[soundtrack],[L'Orchestra Numerique],[Sadness and Sorrow]


,score,item_id_1,item_id_2,name,genres,artists,albums
100299,0.945210,19197327,26388238,Naruto,"[spoken, relax]",[Marco Velocci],"[Naruto Greatest Hits, Nightbook, Naruto]"
100300,0.913779,19197327,10994679,Sadness and Sorrow,[soundtrack],[Piano Squall],[GAME]
100301,0.909214,19197327,22001085,The Sacrifice,"[soundtrack, videogame]","[Aeralie Brighton, Gareth Coker]",[Ori and the Blind Forest]
100302,0.895598,19197327,26413511,Test Drive,"[animated, soundtrack]",[John Powell],"[How To Train Your Dragon, Epic Gaming]"
100303,0.894361,19197327,26413515,Romantic Flight,"[animated, soundtrack]",[John Powell],"[How To Train Your Dragon, Epic Gaming]"
100304,0.891714,19197327,21548569,"Light's Theme (From ""Death Note"")",[soundtrack],[Anime Kei],[L's Themes]
100305,0.888515,19197327,26413508,Forbidden Friendship,"[animated, soundtrack]",[John Powell],[How To Train Your Dragon]
100306,0.886890,19197327,62792370,The Call of the Wild,"[films, soundtrack]",[John Powell],"[Зов предков, The Call of the Wild]"
100307,0.885883,19197327,32374183,Natsuhiboshi,"[spoken, relax]",[Marco Velocci],"[Naruto Greatest Hits, Nightbook, Natsuhiboshi]"
100308,0.885527,19197327,26413509,New Tail,"[animated, soundtrack]",[John Powell],[How To Train Your Dragon]


### Построение ранжирующей модели

Подготовим список кандидатов для обучения ранжирующей модели. В качестве кандидатогенераторов возьмём ранее подготовленные при помощи ALS коллаборативные и контентные рекомендации. Кроме того, добавим 3-й признак - кол-во треков, прослушанных каждым пользователем.

In [6]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("personal_als.parquet")
content_recommendations = pd.read_parquet("content_recommendations.parquet")

In [16]:
# Объединяем списки рекомендаций по совпадению user_id, item_id
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer") 

Оставляем только те строки, где есть оценки от обоих кандидатогенераторов

In [17]:
candidates = candidates[~candidates[['als_score', 'cnt_score']].isnull().any(axis=1)]
candidates

,user_id,item_id,als_score,cnt_score
7956,3243,43013827,0.614118,0.811460
17728,7179,41473,0.567302,0.820208
20097,7846,73495,0.535677,0.867759
21081,8336,15271,0.625788,0.747257
26939,10294,73495,0.822323,0.889507
31656,12014,628383,1.225516,0.791916
31973,12081,73495,0.812035,0.927586
32650,12412,43013827,0.303361,0.787019
54731,21427,73495,0.477418,0.928590
67612,26737,15271,0.651822,0.872216


Добавим в candidates таргеты, используя историю взаимодействий в events_train:
- 1 - для прослушанных item_id (положительный пример),
- 0 - для всех остальных (негативный пример).

В candidates отберем все положительные примеры, а также не менее четырёх негативных для каждого пользователя в положительных примерах.

In [18]:
# Добавляем таргет к кандидатам
events_train["target"] = 1
candidates = candidates.merge(events_train[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], 
                              how='left') 
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
#candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

candidates_for_train = candidates
'''
# для каждого пользователя оставляем только 2 негативных примера
negatives_per_user = 2
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"), 
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])
'''

'\n# для каждого пользователя оставляем только 2 негативных примера\nnegatives_per_user = 2\ncandidates_for_train = pd.concat([\n    candidates_to_sample.query("target == 1"), \n    candidates_to_sample.query("target == 0")         .groupby("user_id")         .apply(lambda x: x.sample(negatives_per_user, random_state=0))\n    ])\n'

Добавим в events_train новый признак пользователей - количество треков, прослушанных каждым пользователем

In [19]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        tracks_played_by_user =("started_at", "count")
    )

    return user_features
    

user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")
candidates_for_train

,user_id,item_id,als_score,cnt_score,target,tracks_played_by_user
0,3243,43013827,0.614118,0.811460,1,1285
1,7179,41473,0.567302,0.820208,0,1459
2,7846,73495,0.535677,0.867759,0,965
3,8336,15271,0.625788,0.747257,1,1684
4,10294,73495,0.822323,0.889507,1,1474


Обучаем модель

In [22]:
# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 'tracks_played_by_user']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6225616	total: 67.3ms	remaining: 1m 7s
100:	learn: 0.0199402	total: 98.2ms	remaining: 874ms
200:	learn: 0.0097198	total: 130ms	remaining: 516ms
300:	learn: 0.0064802	total: 161ms	remaining: 373ms
400:	learn: 0.0048557	total: 193ms	remaining: 288ms
500:	learn: 0.0038809	total: 224ms	remaining: 223ms
600:	learn: 0.0032307	total: 257ms	remaining: 171ms
700:	learn: 0.0027665	total: 289ms	remaining: 123ms
800:	learn: 0.0024250	total: 320ms	remaining: 79.4ms
900:	learn: 0.0022926	total: 350ms	remaining: 38.4ms
999:	learn: 0.0022926	total: 378ms	remaining: 0us


In [23]:
# Сохраняем ранжирующую модель
os.makedirs('models/', exist_ok=True)
cb_model.save_model('models/cb_model.cbm')

Теперь можно получить финальные рекомендации, используя обученную модель.

In [24]:
# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_for_train[candidates_for_train["user_id"].isin(events_test["user_id"].drop_duplicates())]

inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id').cumcount() + 1 

max_recommendations_per_user = 100
recommendations = candidates_to_rank.query('rank <= @max_recommendations_per_user') \
    .rename(columns={"cb_score": "score"}) \
        .drop(columns='target')

In [ ]:
# Смотрим результат
recommendations

In [26]:
# Сохраняем финальные персональные рекомендации в файл
recommendations.to_parquet("recommendations.parquet")

Проведем валидацию финальных рекомендаций.

Посчитаем метрики recall и precision, используя отложенную тестовую выборку, 
а также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [31]:
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_train,
    events_test, 
    recommendations, 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5) 

Common users: 13


In [32]:
print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

precision: 0.000, recall: 0.000


Посмотрим, какие признаки вносят наибольший вклад в ранжирование. 

In [33]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])

feature_importance = feature_importance.sort_values('fi', ascending=False)
feature_importance

,fi
als_score,56.884940
cnt_score,28.081666
tracks_played_by_user,15.033394
